In [64]:
from torch.nn.functional import one_hot
import pandas as pd
import emoji
import torch
import numpy as np

In [41]:
emoji_dictionary = {"0": ":heart:",
                    "1": ":baseball:",
                    "2": ":smile:",
                    "3": ":disappointed:",
                    "4": ":fork_and_knife:"}

In [42]:
def label_to_emoji(target_label):
    assert type(target_label) == str
    emoji_unicode = emoji_dictionary.get(target_label)
    return emoji.emojize(emoji_unicode, use_aliases=True)

In [47]:
def extract_X_Y(file_path):
    data = pd.read_csv(file_path, error_bad_lines=False)
    # In case of prior manilulations.    
    if 'Unnamed: 2' in data.columns or 'Unnamed: 3' in data.columns:
        data = data.drop(columns=['Unnamed: 2', 'Unnamed: 3'])
    
    data.columns = ['phrase', 'label']
    X = data.get('phrase')
    Y = data.get('label')
    return X, Y

In [48]:
X_train, Y_train = extract_X_Y('./data/train_emoji.csv')
X_test, Y_test = extract_X_Y('./data/test_emoji.csv')

### One-Hot

In [65]:
def one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

### One-Hot encoding labels

In [67]:
oh_Y_train = one_hot(Y_train.values, 5)
oh_Y_test = one_hot(Y_test.values, 5)

In [68]:
print("X_train Shape:", X_train.shape)
print("Y_train Shape:", oh_Y_train.shape)
print("X_test Shape:", X_test.shape)
print("Y_test Shape:", oh_Y_test.shape)

X_train Shape: (131,)
Y_train Shape: (131, 5)
X_test Shape: (55,)
Y_test Shape: (55, 5)


In [74]:
idx = 50
print(X_train[idx], label_to_emoji(str(Y_train[idx])))
print("Label index %d is one-hot encoded as:" % Y_train[idx], oh_Y_train[idx])

I think I will end up alone 😞
Label index 3 is one-hot encoded as: [0. 0. 0. 1. 0.]


### Implementing Emojifier V-1